In [13]:
# encoding=utf-8
# Create OpenAI API interface
import os
import sys
from openai import OpenAI
from openai import OpenAIError

# Add project root directory to Python path
sys.path.append('/data/CSY/autodefense/AutoDefense')

# Set proxy environment variables for network access
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

class OpenAI_API(object):
    def __init__(self, api_key, model_name="gpt-3.5-turbo"):
        # Initialize with the standard OpenAI client
        self.client = OpenAI(api_key=api_key)
        self.model_name = model_name
        
    def get_response(self, prompt, max_tokens=500, temperature=0.7, 
                     top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0):
        try:
            # Use the standard chat completion API
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=max_tokens,
                temperature=temperature,
                top_p=top_p,
                frequency_penalty=frequency_penalty,
                presence_penalty=presence_penalty
            )
            return response
        except OpenAIError as e:
            print(f"OpenAI API Error: {e}")
            # Try fallback model if available
            if self.model_name != "gpt-3.5-turbo":
                print("Trying fallback model: gpt-3.5-turbo")
                try:
                    response = self.client.chat.completions.create(
                        model="gpt-3.5-turbo",
                        messages=[
                            {"role": "system", "content": "You are a helpful assistant."},
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=max_tokens,
                        temperature=temperature
                    )
                    return response
                except OpenAIError as fallback_error:
                    print(f"Fallback model also failed: {fallback_error}")
                    raise
            raise

# Main entry point
if __name__ == '__main__':
    try:
        # Get API key from environment or use default
        api_key = os.getenv("OPENAI_API_KEY", "")
        
        # Initialize OpenAI API with the correct model name
        # Note: The standard model name is "gpt-3.5-turbo", not "gpt-35-turbo"
        openai_api = OpenAI_API(api_key, model_name="gpt-3.5-turbo")
        
        # Test query
        prompt = "陈嗣元是少将么"
        print(f"Sending query: {prompt}")
        result = openai_api.get_response(prompt, max_tokens=150)
        
        # Display response
        print(f"Response: {result}")
        if hasattr(result, 'choices') and result.choices:
            print(f"Content: {result.choices[0].message.content}")
        
    except Exception as e:
        print(f"Main program error: {e}")
        print("Possible issues to check:")
        print("1. API key is valid and has access to the model")
        print("2. Network connection and proxy settings are correct")
        print("3. You're using the correct model name (gpt-3.5-turbo is standard)")

Sending query: 陈嗣元是少将么
Response: ChatCompletion(id='chatcmpl-CH6YT0Ls8m0d0Iulyl5N7bpavJDgt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='是的，陈嗣元是中国人民解放军的少将。', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1758192809, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=27, total_tokens=48, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
Content: 是的，陈嗣元是中国人民解放军的少将。
